In [1]:
import pandas as pd
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

In [2]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()
import random

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')
url = "http://localhost:18056"
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

In [3]:
new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", name="test")
user_id = new_user['id']


DEBUG: Making POST request to http://localhost:18056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '259433@example.com', 'name': 'test'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 04 Jun 2025 13:59:14 GMT, Wed, 04 Jun 2025 13:59:14 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '134', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"259433@example.com","name":"test","image_url":null,"max_concurrent_bots":1,"id":6,"created_at":"2025-06-04T13:59:15.107847"}...


In [4]:
user_id

6

In [5]:
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']


DEBUG: Making POST request to http://localhost:18056/admin/users/6/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'token'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 04 Jun 2025 13:59:16 GMT, Wed, 04 Jun 2025 13:59:16 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '113', 'content-type': 'application/json'}
DEBUG: Response content: {"user_id":6,"id":6,"token":"vwpMI9eSezdgEnquLIxnwQtWW83VEt99jVoPgKTE","created_at":"2025-06-04T13:59:17.107738"}...


In [6]:
client = VexaClient(
    base_url=url,
    api_key=user_api_key
)

In [7]:
meeting_url="https://meet.google.com/xce-ctxr-qse"
native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"

In [156]:
meeting_info = client.request_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    bot_name="Vexa",
  #  language='es',
    task = 'transcribe'
)
meeting_id = meeting_info['id']


DEBUG: Making POST request to http://localhost:18056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'vwpMI9eSezdgEnquLIxnwQtWW83VEt99jVoPgKTE'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'xce-ctxr-qse', 'bot_name': 'Vexa', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Wed, 04 Jun 2025 14:21:02 GMT, Wed, 04 Jun 2025 14:21:03 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '392', 'content-type': 'application/json'}
DEBUG: Response content: {"id":16,"user_id":6,"platform":"google_meet","native_meeting_id":"xce-ctxr-qse","constructed_meeting_url":"https://meet.google.com/xce-ctxr-qse","status":"active","bot_container_id":"df8bb65c57e8187b1bf924dccb6541b039b9658226e13eb86f9efda8ea5f9e3f","start_time":"2025-06-04T14:21:03.490548","end_time":null,"created_at":"2025-06-04T14:21:03.288366","updated_at":"2025-06-04T14:21:03.294912"}...


In [193]:
transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
df.sort_values('absolute_start_time').tail(10)


DEBUG: Making GET request to http://localhost:18056/transcripts/google_meet/xce-ctxr-qse
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'vwpMI9eSezdgEnquLIxnwQtWW83VEt99jVoPgKTE'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Wed, 04 Jun 2025 14:23:30 GMT, Wed, 04 Jun 2025 14:23:30 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '6727', 'content-type': 'application/json'}
DEBUG: Response content: {"id":16,"platform":"google_meet","native_meeting_id":"xce-ctxr-qse","constructed_meeting_url":"https://meet.google.com/xce-ctxr-qse","status":"active","start_time":"2025-06-04T14:21:03.490548","end_time":null,"segments":[{"start":0.0,"end":5.0,"text":" Great, we have an excellent conversation, it's all in both.","language":null,"created_at":"2025-06-04T14:23:16.152736","speaker":"info vexa","absolute_start_time":"2025-06-04T14:21:21.571619+00:00","absolute_end_time":"2025-06-04T14:21:26.571619+...


,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
15,68.93,70.05,turn the speakers.,None,None,info vexa,2025-06-04T14:22:30.501619+00:00,2025-06-04T14:22:31.621619+00:00
16,72.44,74.62,That comes forth,None,None,info vexa,2025-06-04T14:22:34.014619+00:00,2025-06-04T14:22:36.194619+00:00
17,72.62,74.62,That comes forth,None,2025-06-04T14:23:16.152842,info vexa,2025-06-04T14:22:34.194619+00:00,2025-06-04T14:22:36.194619+00:00
18,78.94,80.06,But I think that's it for this video.,None,None,info vexa,2025-06-04T14:22:40.515619+00:00,2025-06-04T14:22:41.629619+00:00
19,80.06,86.44,I don't know if it's going to do it the right way or not.,None,None,info vexa,2025-06-04T14:22:41.629619+00:00,2025-06-04T14:22:48.009619+00:00
20,84.94,80.06,Bye.,None,2025-06-04T14:23:16.152612,None,2025-06-04T14:22:46.515619+00:00,2025-06-04T14:22:41.629619+00:00
21,86.44,94.18,the right way or not.,None,None,Dmitry Grankin,2025-06-04T14:22:48.009619+00:00,2025-06-04T14:22:55.749619+00:00
22,94.18,101.54,"Yeah, just because I'm now switched to Dimitri Yerankin and it's...",None,None,Dmitry Grankin,2025-06-04T14:22:55.749619+00:00,2025-06-04T14:23:03.109619+00:00
23,108.38,122.31,yeah so it can make it can make errors with this identification job but at,None,None,Dmitry Grankin,2025-06-04T14:23:09.950619+00:00,2025-06-04T14:23:23.880619+00:00
24,122.31,127.31,At least it outputs the speaker.,None,None,Dmitry Grankin,2025-06-04T14:23:23.880619+00:00,2025-06-04T14:23:28.880619+00:00


In [155]:
client.stop_bot(platform="google_meet",native_meeting_id=native_meeting_id)


DEBUG: Making DELETE request to http://localhost:18056/bots/google_meet/xce-ctxr-qse
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'vwpMI9eSezdgEnquLIxnwQtWW83VEt99jVoPgKTE'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Wed, 04 Jun 2025 14:20:19 GMT, Wed, 04 Jun 2025 14:20:20 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...


{'message': 'Stop request accepted and is being processed.'}

In [8]:
user_id

2

In [309]:
running_bots = client.get_running_bots_status()


DEBUG: Making GET request to http://localhost:18056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': '5IRfzrEVsU2T8VBq0oNIFA1uIddTbAlUiiFaesTq'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Tue, 27 May 2025 22:35:23 GMT, Tue, 27 May 2025 22:35:23 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '19', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[]}...


In [32]:
admin_client.update_user(user_id,max_concurrent_bots=3)


DEBUG: Making PATCH request to http://localhost:8056/admin/users/2
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'lkwnekf68&IJBKb7'}
DEBUG: Params: None
DEBUG: JSON data: {'max_concurrent_bots': 3}
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Tue, 22 Apr 2025 13:27:45 GMT, Tue, 22 Apr 2025 13:27:45 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '134', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"914545@example.com","name":"test","image_url":null,"max_concurrent_bots":3,"id":2,"created_at":"2025-04-22T13:17:30.891747"}...


{'email': '914545@example.com',
 'name': 'test',
 'image_url': None,
 'max_concurrent_bots': 3,
 'id': 2,
 'created_at': '2025-04-22T13:17:30.891747'}

In [30]:
client.update_bot_config(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    language='en',
    task = 'transcribe'
)


DEBUG: Making PUT request to http://localhost:8056/bots/google_meet/hdn-ubcg-jkq/config
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'iDwbpzUxW5QCmmKRsaN04yZUnz9LYow56lOTa4Rg'}
DEBUG: Params: None
DEBUG: JSON data: {'language': 'en', 'task': 'transcribe'}
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Tue, 22 Apr 2025 11:42:21 GMT, Tue, 22 Apr 2025 11:42:21 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '67', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Reconfiguration request accepted and sent to the bot."}...


{'message': 'Reconfiguration request accepted and sent to the bot.'}

In [ ]:
running_bots = client.get_running_bots_status()


DEBUG: Making GET request to http://localhost:8056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'yl9x4xUZdXnLZsRxfj6A7IQJEyf2czFpQBjV1e93'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Tue, 22 Apr 2025 13:58:29 GMT, Tue, 22 Apr 2025 13:58:29 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1202', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"e9c6def1ea1468b67b5bb8c010eff03febadf40e8b1727d6dadf8a72ddeb9c9a","container_name":"vexa-bot-11-4f4fd204","platform":"google_meet","native_meeting_id":"phu-przh-ove","status":"Up 30 minutes","created_at":"2025-04-22T13:27:49+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"2"},"meeting_id_from_name":"11"},{"container_id":"433f2e1fd9776f6ddae1262b530b62b66b5a3bca16be9b25bfe1a7b10f3c3967","con...


In [ ]:
running_bots = client.get_running_bots_status()
for running_bot in running_bots:
    client.stop_bot(platform=running_bot['platform'],native_meeting_id=running_bot['native_meeting_id'])


DEBUG: Making GET request to http://localhost:8056/bots/status
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'yl9x4xUZdXnLZsRxfj6A7IQJEyf2czFpQBjV1e93'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Tue, 22 Apr 2025 13:58:29 GMT, Tue, 22 Apr 2025 13:58:29 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '1202', 'content-type': 'application/json'}
DEBUG: Response content: {"running_bots":[{"container_id":"e9c6def1ea1468b67b5bb8c010eff03febadf40e8b1727d6dadf8a72ddeb9c9a","container_name":"vexa-bot-11-4f4fd204","platform":"google_meet","native_meeting_id":"phu-przh-ove","status":"Up 30 minutes","created_at":"2025-04-22T13:27:49+00:00","labels":{"org.opencontainers.image.ref.name":"ubuntu","org.opencontainers.image.version":"22.04","vexa.user_id":"2"},"meeting_id_from_name":"11"},{"container_id":"433f2e1fd9776f6ddae1262b530b62b66b5a3bca16be9b25bfe1a7b10f3c3967","con...


In [298]:
meetings = client.get_meetings()


DEBUG: Making GET request to http://localhost:8056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'NvsJRMspflhJ8qRZ5IQiZDrfkdsXyqncuaVlvKoR'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Mon, 21 Apr 2025 15:49:02 GMT, Mon, 21 Apr 2025 15:49:02 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '406', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":6,"user_id":2,"platform":"google_meet","native_meeting_id":"fsn-apcm-nqf","constructed_meeting_url":"https://meet.google.com/fsn-apcm-nqf","status":"active","bot_container_id":"71d0469f81097eb209f27ab3ecfad10f80c93bf010eb9a8ed982f01122eda6ac","start_time":"2025-04-21T15:48:34.986970","end_time":null,"created_at":"2025-04-21T15:48:34.750718","updated_at":"2025-04-21T15:48:34.757661"}]}...
